In [1]:
from numpy import genfromtxt
import os
import pandas as pd
import numpy as np
import gc
from scipy import stats
pd.set_option('display.max_colwidth', -1)

<ipython-input-1-f0905c640c3e>:7: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [2]:
gc.collect()
survey_path = os.path.join(os.path.abspath(''),'surveys')
surveys = {}
for j,survey in enumerate(os.listdir(survey_path)):
    if 'Participant' in survey:
        continue
    print(survey)
    user_id = survey.split(sep=' ')[1]
    survey_file = os.path.join(survey_path,survey)
    df = pd.read_csv(survey_file,dtype='str')
    experiments = []
    for i,sl in enumerate([slice(6,17),slice(18,29), slice(30,41), slice(43,54), slice(55,66), slice(67,78)]):
        exp_df = df.iloc[sl]
        exp_df = exp_df.rename(columns={x:y for x,y in zip(df.columns,range(0,len(df.columns)))})
        exp_df = exp_df.fillna(0)
        exp_data = [exp_df.iloc[0][0], exp_df.iloc[1:-2].set_index(0), exp_df.iloc[-2:]]
        exp_data[1] = exp_data[1].replace('x',1)
        exp_data[1] = exp_data[1].iloc[:,list(range(7))]
        exp_data[1] = exp_data[1].astype(int)
        exp_data[1] = exp_data[1].dot(range(1,8))
        exp_data[1] = pd.DataFrame(exp_data[1]).transpose()
        experiments.append(exp_data)

    exp,ratings,comments = zip(*experiments)
    rate_df = pd.concat(ratings)
    rate_df_ls, rate_df_b = rate_df[:3],rate_df[3:]
    rate_df_ls = pd.DataFrame(rate_df_ls.reset_index(drop=True))
    rate_df_b = pd.DataFrame(rate_df_b.reset_index(drop=True))
    surveys[user_id] = {
                    'experiment names': exp,
                    'all ratings': rate_df.copy(),
                    'treated lightswitch ratings': rate_df_ls.copy(),
                    'treated bottle ratings': rate_df_b.copy(),
                    'comments': pd.concat(comments).copy()
                        }

User 0 - Sheet1.csv
User 1 - Sheet1.csv
User 2 - Sheet1.csv
User 3 - Sheet1.csv
User 4 - Sheet1.csv
User 5 - Sheet1.csv
User 9 - Sheet1.csv


In [3]:
survey_file = os.path.join(survey_path,'Participant Info - Sheet1.csv')
df = pd.read_csv(survey_file,dtype='str')
df = df[1:13][['User ID','Condition']]
df = df.set_index('User ID')
user_cond = df.to_dict()['Condition']
letter_number = {'A': 0, 'B': 1, 'C': 2}
for k,v in user_cond.items():
    order = list(v)
    order = {letter_number[letter]:i for i,letter in enumerate(order)}
    order = [order[i] for i in range(3)]
    user_cond[k] = order

In [4]:
for k in surveys.keys():
    surveys[k]['lightswitch ratings'] = surveys[k]['treated lightswitch ratings']
    surveys[k]['lightswitch ratings'] = surveys[k]['lightswitch ratings'].reindex(user_cond[k])
    surveys[k]['lightswitch ratings'] = surveys[k]['lightswitch ratings'].reset_index(drop=True)
    surveys[k]['bottle ratings'] = surveys[k]['treated bottle ratings'].reindex(user_cond[k])
    surveys[k]['bottle ratings'] = surveys[k]['bottle ratings'].reindex(user_cond[k])
    surveys[k]['bottle ratings'] = surveys[k]['bottle ratings'].reset_index(drop=True)

In [5]:
print(surveys['9']['lightswitch ratings'].transpose())

                                                                                                                           0  \
0                                                                                                                              
The system performed the task I wanted                                                                                     5   
I felt in control                                                                                                          6   
The system responded to my input in the way that I expected                                                                6   
The system was competent at performing tasks, even if they weren't the tasks I wanted                                      6   
The system improved over time                                                                                              2   
I improved at using the system over time                                                                

In [7]:
ls_ratings = pd.concat([survey['lightswitch ratings'] for survey in surveys.values()])
b_ratings = pd.concat([survey['bottle ratings'] for survey in surveys.values()])

def make_mean_sd_table(ls_ratings):
    ls_ratings.index.name = 'Treatment'
    ls_ratings.columns.name = 'Question'
    index_mapper = {0: 'Baseline', 1:'ASHA', 2:'Our Method + Shift'}

    ls_ratings = ls_ratings.rename(index=index_mapper)
    ls_ratings = ls_ratings.groupby(by='Treatment')
    ls_ratings_mean = ls_ratings.mean()
    ls_ratings_mean = ls_ratings_mean.transpose()
    ls_ratings_sd = ls_ratings.std()
    ls_ratings_sd = ls_ratings_sd.transpose()
    ls_ratings_count = ls_ratings.count()
    ls_ratings_count = ls_ratings_count.transpose()
    
    ls_ratings = ls_ratings_mean.copy().round(1)

    ls_ratings_mean.columns = pd.MultiIndex.from_tuples([(col,'mean') for col in ls_ratings_mean.columns],names=['Treatment','Metric'])
    ls_ratings_sd.columns = pd.MultiIndex.from_tuples([(col,'sd') for col in ls_ratings_sd.columns],names=['Treatment','Metric'])
    ls_ratings_count.columns = pd.MultiIndex.from_tuples([(col,'count') for col in ls_ratings_count.columns],names=['Treatment','Metric'])
    ls_ratings_metrics = pd.concat([ls_ratings_mean,ls_ratings_sd,ls_ratings_count],axis=1)
    ls_ratings_metrics = ls_ratings_metrics.sort_index(axis=1)
    ttest = lambda r: stats.ttest_ind_from_stats(r['Baseline','mean'],r['Baseline','sd'],r['Baseline','count'],
                                                 r['ASHA','mean'],r['ASHA','sd'],r['ASHA','count'])[1]
    ls_ratings['p'] = ls_ratings_metrics.apply(ttest,axis=1).round(2)
#     print(ls_ratings)
#     print(ls_ratings.to_latex())
    return ls_ratings
    
ls_ratings = make_mean_sd_table(ls_ratings)
b_ratings = make_mean_sd_table(b_ratings)
ls_ratings = ls_ratings[['p','Baseline','ASHA']]
b_ratings = b_ratings[['p','Baseline','ASHA']]
ls_ratings.columns = pd.MultiIndex.from_tuples([('LightSwitch',col) for col in ls_ratings.columns],names=['Environment','Treatment'])
b_ratings.columns = pd.MultiIndex.from_tuples([('Bottle',col) for col in b_ratings.columns],names=['Environment','Treatment'])

ratings = pd.concat([ls_ratings,b_ratings],axis=1)
ratings = ratings.sort_index(axis=1)
ratings.index.name = None
ratings.columns.names = [None,None]
print(ratings.to_latex(multicolumn_format='c'))
print(ratings)

\begin{tabular}{lrrrrrr}
\toprule
{} & \multicolumn{3}{c}{Bottle} & \multicolumn{3}{c}{LightSwitch} \\
{} &   ASHA & Baseline &     p &        ASHA & Baseline &     p \\
\midrule
The system performed the task I wanted                                                                                    &  4.7 &  4.0 &  0.46 &  4.6 &  3.4 &  0.10 \\
I felt in control                                                                                                         &  3.9 &  3.0 &  0.34 &  3.9 &  3.1 &  0.41 \\
The system responded to my input in the way that I expected                                                               &  4.1 &  3.3 &  0.39 &  3.6 &  3.7 &  0.89 \\
The system was competent at performing tasks, even if they weren't the tasks I wanted                                     &  5.6 &  4.9 &  0.40 &  5.4 &  4.6 &  0.18 \\
The system improved over time                                                                                             &  4.7 &  3.3 &  0.12 &